# Tests de funcionalidad de la API de series de tiempo

In [125]:
import os
import pandas as pd
import numpy as np
import requests
from io import StringIO

## Variables

**Obligatorio**: setear la variable de entorno API_URL o setear la variable BASE_URL al ambiente de la API que se quiere probar

In [ ]:
BASE_URL = ""
METADATA_URL = ''
ENDPOINT_URL = BASE_URL + 'series/api/series/'

In [132]:
series_metadata = pd.read_csv(METADATA_URL)
series_metadata.tail(5)

,catalogo_id,dataset_id,distribucion_id,serie_id,indice_tiempo_frecuencia,serie_titulo,serie_unidades,serie_descripcion,distribucion_titulo,distribucion_descripcion,...,dataset_descripcion,dataset_tema,serie_indice_inicio,serie_indice_final,serie_valores_cant,serie_dias_no_cubiertos,serie_actualizada,serie_valor_ultimo,serie_valor_anterior,serie_var_pct_anterior
19978,sspm,99,99.3,99.3_ING_2008_0_17,R/P1M,ipc_nivel_general,Índice abr-2008=100,Índice de precios al consumidor nivel general....,"Índice de Precios al Consumidor, por grupos. D...",Subsecretaría de Programación Macroeconómica.,...,Índice de Precios al Consumidor del Gran Bueno...,Precios,2006-12-01,2013-12-01,85.0,1834.0,False,166.840000,164.510000,0.014163
19979,sspm,99,99.3,99.3_IR_2008_0_13,R/P1M,ipc_regulados,Índice abr-2008=100,Índice de precios al consumidor regulados. Val...,"Índice de Precios al Consumidor, por grupos. D...",Subsecretaría de Programación Macroeconómica.,...,Índice de Precios al Consumidor del Gran Bueno...,Precios,2006-12-01,2013-12-01,85.0,1834.0,False,161.040000,159.290000,0.010986
19980,sspm,99,99.3,99.3_IR_2008_0_9,R/P1M,ipc_resto,Índice abr-2008=100,Índice de precios al consumidor IPC resto. Val...,"Índice de Precios al Consumidor, por grupos. D...",Subsecretaría de Programación Macroeconómica.,...,Índice de Precios al Consumidor del Gran Bueno...,Precios,2006-12-01,2013-12-01,85.0,1834.0,False,161.740000,159.250000,0.015636
19981,sspm,99,99.3,99.3_PREIR_2008_0_40,R/P1M,precios_relativos_estacionales_ipc_resto,Índice abr-2008=100,Índice de precios al consumidor precios relati...,"Índice de Precios al Consumidor, por grupos. D...",Subsecretaría de Programación Macroeconómica.,...,Índice de Precios al Consumidor del Gran Bueno...,Precios,2006-12-01,2013-12-01,85.0,1834.0,False,137.591196,138.411303,-0.005925
19982,sspm,99,99.3,99.3_PRRIR_2008_0_37,R/P1M,precios_relativos_regulados_ipc_resto,Índice abr-2008=100,Índice de precios al consumidor precios relati...,"Índice de Precios al Consumidor, por grupos. D...",Subsecretaría de Programación Macroeconómica.,...,Índice de Precios al Consumidor del Gran Bueno...,Precios,2006-12-01,2013-12-01,85.0,1834.0,False,99.567207,100.025118,-0.004578


In [133]:
series_metadata.serie_id.count()

19983

## Chequeo de todas las series 

Le pegamos al endpoint para todas las series y verificamos que la API devuelve una respuesta satisfactoria (status code 200). Contamos la cantidad de casos satisfactorios (True) y no (False)

In [134]:
def api_series_head(serie_id):
    return requests.head(ENDPOINT_URL, params={'ids': serie_id}).status_code == 200

### Cantidad de respuestas válidas

In [135]:
valid_responses = series_metadata.serie_id[:10].apply(api_series_head)
valid_responses.value_counts()

True     9
False    1
Name: serie_id, dtype: int64

### En porcentajes

In [136]:
valid_responses.value_counts().apply(lambda x: x/len(valid_responses))

True     0.9
False    0.1
Name: serie_id, dtype: float64

## Chequeo de modos de representación

In [138]:
import random

def get_source_csv(serie):
    #response = requests.get(ENDPOINT_URL, params={'ids': serie, 'metadata': 'only'}).json()
    #distribution_url = response['meta'][1]['distribution']['downloadURL']
    distribution_url = series_metadata[series_metadata.serie_id == serie]["distribucion_url_descarga"].iloc[0]
    
    #title = response['meta'][1]['field']['title']
    title = series_metadata[series_metadata.serie_id == serie]["serie_titulo"].iloc[0]

    orig_csv = pd.read_csv(distribution_url, parse_dates=['indice_tiempo'], index_col='indice_tiempo')

    return orig_csv, title

### Serie original

In [68]:
# toma un sample de una serie
serie = series_metadata.serie_id.sample(1)

serie_idx = serie.index[0]
serie = serie.values[0]
serie = "20.2_FME_1997_0_30"
serie = "168.1_T_CAMBIOR_D_0_0_26"

In [69]:
orig_csv, title = get_source_csv(serie)

### Serie de la API: valor original

In [70]:
api_csv, status_code = api_call(serie)

print("Status code:", status_code)
api_csv.head(1)

Status code: 200


,tipo_cambio_bna_vendedor
indice_tiempo,
2016-03-19,15.1


In [71]:
#orig_csv = orig_csv[:len(api_csv)]
orig_csv = orig_csv[-len(api_csv):]
equality_check = np.isclose(orig_csv[title], api_csv[title], equal_nan=True)
equal_df = pd.Series(equality_check)

equal_df.value_counts()

False    833
True     167
dtype: int64

### Chequeo de los datos de todas las series

Chequea que los datos que devuelve la API para cada una de las series sean correctos respecto del CSV original del que se toman.

In [139]:
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures

In [140]:
def chequear_serie(serie_id):
    """Test completo de chequear una serie contra su CSV original."""
    
    try:
        orig_csv, title = get_source_csv(serie_id)
    except Exception as e:
        return {}, repr(e)
    
    api_csv, status_code = api_call(serie_id)
    
    if isinstance(api_csv, pd.DataFrame):
        col = orig_csv[[title]]
        df = pd.merge(col, api_csv, 
                      left_index=True, right_index=True, how="inner"
        )
        equality_check = np.isclose(df[title + "_x"], df[title + "_y"], 
                                    rtol=0.001, equal_nan=True)
        equal_df = pd.Series(equality_check)

        return equal_df.value_counts(), status_code

    else:
        return {}, status_code

In [141]:
print(chequear_serie("168.1_T_CAMBIOR_D_0_0_26"))
print(chequear_serie("20.2_FME_1997_0_30"))
print(chequear_serie("64.1_GTTV_0_0_23"))

(True    1000
dtype: int64, 200)
(True    3
dtype: int64, 200)
(Series([], dtype: int64), 200)


In [142]:
def chequear_series(series):
    """Testea una lista de series concurrentemente, permitiendo frenar 
    la ejecución con el teclado y obtener resultados parciales."""
    
    results = []

    def chequear_serie_bulk(serie_id):
        res, status_code = chequear_serie(serie_id)
        results.append({
            "serie_id": serie_id, "ok": res.get(True, 0), 
            "error": res.get(False, 0), "status_code": status_code
        })
        s = "Tests realizados: {}".format(len(results))
        print(s)
    
    try:
        with ThreadPoolExecutor(max_workers=10) as executor:
            futures = []
            for serie_id in series:
                fs = executor.submit(chequear_serie_bulk, serie_id)
                futures.append(fs)
            concurrent.futures.wait(futures)
            
    except KeyboardInterrupt:
        return results
    
    return results

In [143]:
series = series_metadata[series_metadata.catalogo_id.isin(["sspm", "energia", "siep"])].serie_id
results = chequear_series(series)
df_results = pd.DataFrame(results)

Tests realizados: 1
Tests realizados: 2
Tests realizados: 3
Tests realizados: 4
Tests realizados: 5
Tests realizados: 6
Tests realizados: 7
Tests realizados: 8
Tests realizados: 9
Tests realizados: 10
Tests realizados: 11
Tests realizados: 12
Tests realizados: 13
Tests realizados: 14
Tests realizados: 15
Tests realizados: 16
Tests realizados: 17
Tests realizados: 18
Tests realizados: 19
Tests realizados: 20
Tests realizados: 21
Tests realizados: 22
Tests realizados: 23
Tests realizados: 24
Tests realizados: 25
Tests realizados: 26
Tests realizados: 27Tests realizados: 28

Tests realizados: 29
Tests realizados: 30
Tests realizados: 31
Tests realizados: 32
Tests realizados: 33
Tests realizados: 34
Tests realizados: 35
Tests realizados: 36
Tests realizados: 37
Tests realizados: 38
Tests realizados: 39
Tests realizados: 40
Tests realizados: 41
Tests realizados: 42
Tests realizados: 43
Tests realizados: 44
Tests realizados: 45
Tests realizados: 46
Tests realizados: 47
Tests realizados: 48
T

Tests realizados: 381
Tests realizados: 382
Tests realizados: 383
Tests realizados: 384
Tests realizados: 385
Tests realizados: 386
Tests realizados: 387
Tests realizados: 388
Tests realizados: 389
Tests realizados: 390
Tests realizados: 391
Tests realizados: 392
Tests realizados: 393
Tests realizados: 394
Tests realizados: 395
Tests realizados: 396
Tests realizados: 397
Tests realizados: 398
Tests realizados: 399
Tests realizados: 400
Tests realizados: 401
Tests realizados: 402
Tests realizados: 403
Tests realizados: 404
Tests realizados: 405
Tests realizados: 406
Tests realizados: 407
Tests realizados: 408
Tests realizados: 409
Tests realizados: 410
Tests realizados: 411
Tests realizados: 412
Tests realizados: 413
Tests realizados: 414
Tests realizados: 415
Tests realizados: 416
Tests realizados: 417
Tests realizados: 418
Tests realizados: 419
Tests realizados: 420
Tests realizados: 421
Tests realizados: 422
Tests realizados: 423
Tests realizados: 424
Tests realizados: 425
Tests real

Tests realizados: 761
Tests realizados: 762
Tests realizados: 763
Tests realizados: 764
Tests realizados: 765
Tests realizados: 766
Tests realizados: 767
Tests realizados: 768
Tests realizados: 769
Tests realizados: 770
Tests realizados: 771
Tests realizados: 772
Tests realizados: 773
Tests realizados: 774
Tests realizados: 775
Tests realizados: 776
Tests realizados: 777
Tests realizados: 778
Tests realizados: 779
Tests realizados: 780
Tests realizados: 781
Tests realizados: 782
Tests realizados: 783
Tests realizados: 784
Tests realizados: 785
Tests realizados: 786
Tests realizados: 787
Tests realizados: 788
Tests realizados: 789Tests realizados: 790

Tests realizados: 791
Tests realizados: 792
Tests realizados: 793
Tests realizados: 794
Tests realizados: 795
Tests realizados: 796
Tests realizados: 797
Tests realizados: 798
Tests realizados: 799
Tests realizados: 800
Tests realizados: 801
Tests realizados: 802
Tests realizados: 803
Tests realizados: 804
Tests realizados: 805
Tests real

Tests realizados: 1130
Tests realizados: 1131
Tests realizados: 1132
Tests realizados: 1133
Tests realizados: 1134
Tests realizados: 1135
Tests realizados: 1136
Tests realizados: 1137
Tests realizados: 1138
Tests realizados: 1139
Tests realizados: 1140
Tests realizados: 1141
Tests realizados: 1142
Tests realizados: 1143
Tests realizados: 1144
Tests realizados: 1145
Tests realizados: 1146
Tests realizados: 1147
Tests realizados: 1148
Tests realizados: 1149
Tests realizados: 1150
Tests realizados: 1151
Tests realizados: 1152
Tests realizados: 1153
Tests realizados: 1154
Tests realizados: 1155
Tests realizados: 1156
Tests realizados: 1157
Tests realizados: 1158
Tests realizados: 1159
Tests realizados: 1160
Tests realizados: 1161
Tests realizados: 1162
Tests realizados: 1163
Tests realizados: 1164
Tests realizados: 1165
Tests realizados: 1166
Tests realizados: 1167
Tests realizados: 1168
Tests realizados: 1169
Tests realizados: 1170
Tests realizados: 1171
Tests realizados: 1172
Tests reali

Tests realizados: 1488
Tests realizados: 1489
Tests realizados: 1490
Tests realizados: 1491
Tests realizados: 1492
Tests realizados: 1493
Tests realizados: 1494
Tests realizados: 1495
Tests realizados: 1496
Tests realizados: 1497
Tests realizados: 1498
Tests realizados: 1499
Tests realizados: 1500
Tests realizados: 1501
Tests realizados: 1502
Tests realizados: 1503
Tests realizados: 1504
Tests realizados: 1505
Tests realizados: 1506
Tests realizados: 1507
Tests realizados: 1508
Tests realizados: 1509
Tests realizados: 1510
Tests realizados: 1511
Tests realizados: 1512
Tests realizados: 1513
Tests realizados: 1514
Tests realizados: 1515
Tests realizados: 1516
Tests realizados: 1517
Tests realizados: 1518
Tests realizados: 1519
Tests realizados: 1520
Tests realizados: 1521
Tests realizados: 1522
Tests realizados: 1523
Tests realizados: 1524
Tests realizados: 1525
Tests realizados: 1526
Tests realizados: 1527
Tests realizados: 1528
Tests realizados: 1529
Tests realizados: 1530
Tests reali

...

Tests realizados: 18402
Tests realizados: 18403
Tests realizados: 18404
Tests realizados: 18405
Tests realizados: 18406
Tests realizados: 18407
Tests realizados: 18408
Tests realizados: 18409Tests realizados: 18410

Tests realizados: 18411
Tests realizados: 18412
Tests realizados: 18413
Tests realizados: 18414
Tests realizados: 18415
Tests realizados: 18416
Tests realizados: 18417
Tests realizados: 18418
Tests realizados: 18419
Tests realizados: 18420
Tests realizados: 18421
Tests realizados: 18422
Tests realizados: 18423
Tests realizados: 18424
Tests realizados: 18425
Tests realizados: 18426
Tests realizados: 18427
Tests realizados: 18428
Tests realizados: 18429
Tests realizados: 18430
Tests realizados: 18431
Tests realizados: 18432
Tests realizados: 18433
Tests realizados: 18434
Tests realizados: 18435
Tests realizados: 18436
Tests realizados: 18437
Tests realizados: 18438
Tests realizados: 18439
Tests realizados: 18440
Tests realizados: 18441
Tests realizados: 18442
Tests realizados

Tests realizados: 18748
Tests realizados: 18749
Tests realizados: 18750
Tests realizados: 18751
Tests realizados: 18752
Tests realizados: 18753
Tests realizados: 18754
Tests realizados: 18755
Tests realizados: 18756
Tests realizados: 18757
Tests realizados: 18758
Tests realizados: 18759
Tests realizados: 18760
Tests realizados: 18761
Tests realizados: 18762
Tests realizados: 18763
Tests realizados: 18764
Tests realizados: 18765
Tests realizados: 18766
Tests realizados: 18767
Tests realizados: 18768
Tests realizados: 18769
Tests realizados: 18770
Tests realizados: 18771
Tests realizados: 18772
Tests realizados: 18773
Tests realizados: 18774
Tests realizados: 18775
Tests realizados: 18776
Tests realizados: 18777
Tests realizados: 18778
Tests realizados: 18779
Tests realizados: 18780
Tests realizados: 18781
Tests realizados: 18782
Tests realizados: 18783
Tests realizados: 18784
Tests realizados: 18785
Tests realizados: 18786
Tests realizados: 18787
Tests realizados: 18788
Tests realizados

Tests realizados: 19097
Tests realizados: 19098
Tests realizados: 19099
Tests realizados: 19100
Tests realizados: 19101
Tests realizados: 19102
Tests realizados: 19103
Tests realizados: 19104
Tests realizados: 19105
Tests realizados: 19106
Tests realizados: 19107
Tests realizados: 19108
Tests realizados: 19109
Tests realizados: 19110
Tests realizados: 19111
Tests realizados: 19112
Tests realizados: 19113
Tests realizados: 19114
Tests realizados: 19115
Tests realizados: 19116
Tests realizados: 19117
Tests realizados: 19118
Tests realizados: 19119
Tests realizados: 19120
Tests realizados: 19121
Tests realizados: 19122
Tests realizados: 19123
Tests realizados: 19124
Tests realizados: 19125
Tests realizados: 19126
Tests realizados: 19127
Tests realizados: 19128
Tests realizados: 19129
Tests realizados: 19130
Tests realizados: 19131
Tests realizados: 19132
Tests realizados: 19133
Tests realizados: 19134
Tests realizados: 19135
Tests realizados: 19136
Tests realizados: 19137
Tests realizados

Tests realizados: 19440
Tests realizados: 19441
Tests realizados: 19442
Tests realizados: 19443
Tests realizados: 19444
Tests realizados: 19445
Tests realizados: 19446
Tests realizados: 19447
Tests realizados: 19448
Tests realizados: 19449
Tests realizados: 19450
Tests realizados: 19451
Tests realizados: 19452
Tests realizados: 19453
Tests realizados: 19454
Tests realizados: 19455
Tests realizados: 19456
Tests realizados: 19457
Tests realizados: 19458
Tests realizados: 19459
Tests realizados: 19460
Tests realizados: 19461
Tests realizados: 19462
Tests realizados: 19463
Tests realizados: 19464
Tests realizados: 19465
Tests realizados: 19466
Tests realizados: 19467
Tests realizados: 19468
Tests realizados: 19469
Tests realizados: 19470
Tests realizados: 19471
Tests realizados: 19472
Tests realizados: 19473
Tests realizados: 19474
Tests realizados: 19475
Tests realizados: 19476
Tests realizados: 19477
Tests realizados: 19478
Tests realizados: 19479
Tests realizados: 19480
Tests realizados

Tests realizados: 19783
Tests realizados: 19784
Tests realizados: 19785
Tests realizados: 19786
Tests realizados: 19787
Tests realizados: 19788
Tests realizados: 19789
Tests realizados: 19790
Tests realizados: 19791
Tests realizados: 19792
Tests realizados: 19793Tests realizados: 19794

Tests realizados: 19795
Tests realizados: 19796Tests realizados: 19797

Tests realizados: 19798
Tests realizados: 19799
Tests realizados: 19800
Tests realizados: 19801
Tests realizados: 19802
Tests realizados: 19803
Tests realizados: 19804
Tests realizados: 19805


In [144]:
len(series)

19821

In [145]:
len(df_results)

19805

In [146]:
df_results = df_results.fillna(0)
df_results["error_pct"] = df_results.error / (df_results.error + df_results.ok)

In [147]:
df_results.to_csv("errores-series-valores-prod.csv", encoding="utf8", index=False)

In [148]:
results_filter = df_results[
    (df_results.error_pct > 0.0001) & 
    (df_results.error > 0.0)
].sort_values("error_pct", ascending=False)

results_filter.to_csv("errores-series-valores-prod-deshabilitar.csv", encoding="utf8", index=False)

results_filter

0

### Cambio absoluto

In [16]:
api_csv = api_call(serie, representation_mode='change')

In [17]:
orig_csv_change, title = get_source_csv(serie)
orig_csv_change[title] = orig_csv_change[title].diff(1)
orig_csv_change = orig_csv_change[:len(api_csv)]

In [18]:
equality_check = np.isclose(orig_csv_change[title], api_csv[title], equal_nan=True)
equal_df = pd.Series(equality_check)

equal_df.value_counts()

True    56
dtype: int64

### Cambio porcentual

In [19]:
api_csv = api_call(serie, representation_mode='percent_change')

In [20]:
orig_csv_pct_change, title = get_source_csv(serie)
orig_csv_pct_change[title] = orig_csv_pct_change[title].pct_change()
orig_csv_pct_change = orig_csv_pct_change[:len(api_csv)]

In [21]:
equality_check = np.isclose(orig_csv_pct_change[title], api_csv[title], equal_nan=True)
equal_df = pd.Series(equality_check)

equal_df.value_counts()

True    56
dtype: int64

### Colapsos de datos

Aplico las agregaciones máximo y mínimos de la API, y también al CSV original con pandas. Comparo los resultados

In [22]:
api_max = api_call(serie, collapse='year', collapse_aggregation='max')
api_max['api_max'] = api_max[title]
del api_max[title]
api_call(serie).resample('AS').apply(max).join(api_max)

,gtos_cap_transf_cap_ot_1993_2006,api_max
indice_tiempo,,
1993-01-01,16.0,16.0
1994-01-01,14.0,14.0
1995-01-01,33.7,33.7
1996-01-01,7.7,7.7
1997-01-01,21.4,21.4
1998-01-01,19.1,19.1
1999-01-01,16.5,16.5
2000-01-01,14.6,14.6
2001-01-01,9.6,9.6


In [23]:
api_max = api_call(serie, collapse='year', collapse_aggregation='min')
api_max['api_min'] = api_max[title]
del api_max[title]
api_call(serie).resample('AS').apply(min).join(api_max)

,gtos_cap_transf_cap_ot_1993_2006,api_min
indice_tiempo,,
1993-01-01,5.1,5.1
1994-01-01,8.7,8.7
1995-01-01,2.8,2.8
1996-01-01,3.5,3.5
1997-01-01,11.6,11.6
1998-01-01,11.0,11.0
1999-01-01,3.8,3.8
2000-01-01,1.2,1.2
2001-01-01,5.9,5.9


### Metadata

In [24]:
orig_meta = series_metadata.iloc[serie_idx]
orig_meta

catalogo_id                                                               sspm
dataset_id                                                                 375
distribucion_id                                                          375.4
serie_id                                            375.4_GTOS_CAP_T006__32_51
indice_tiempo_frecuencia                                                 R/P3M
serie_titulo                                  gtos_cap_transf_cap_ot_1993_2006
serie_unidades                                               Millones de pesos
serie_descripcion            Gastos de capital transferencias de capital ot...
distribucion_titulo          Organismos Descentralizados. Valores trimestra...
distribucion_descripcion     Esquema Ahorro - Inversión - Financiamiento. O...
distribucion_url_descarga    http://infra.datos.gob.ar/catalog/sspm/dataset...
dataset_responsable              Subsecretaría de Programación Macroeconómica.
dataset_fuente                                      

In [25]:
metadata = requests.get(ENDPOINT_URL, params={'ids': serie, 'metadata': 'full'}).json()

In [26]:
# Metadatos del índice de tiempo
metadata['meta'][0]

{'frequency': 'quarter', 'start_date': '1993-01-01', 'end_date': '2006-10-01'}

In [27]:
serie_meta = metadata['meta'][1]
serie_meta

{'catalog': {'publisher': {'mbox': 'datoseconomicos@mecon.gov.ar',
   'name': 'Subsecretaría de Programación Macroeconómica.'},
  'license': 'Creative Commons Attribution 4.0',
  'description': 'Catálogo de datos abiertos de la Subsecretaría de Programación Macroeconómica.',
  'language': ['SPA'],
  'superThemeTaxonomy': 'http://datos.gob.ar/superThemeTaxonomy.json',
  'issued': '2017-09-28',
  'rights': '2017-09-28',
  'modified': '2017-09-28',
  'spatial': 'ARG',
  'title': 'Datos Programación Macroeconómica',
  'identifier': 'sspm'},
 'dataset': {'publisher': {'mbox': 'datoseconomicos@mecon.gov.ar',
   'name': 'Subsecretaría de Programación Macroeconómica.'},
  'landingPage': 'http://www.minhacienda.gob.ar/secretarias/politica-economica/programacion-macroeconomica/',
  'keyword': ['Información Económica al Día', 'Finanzas Públicas'],
  'superTheme': ['ECON'],
  'title': 'Esquema Ahorro - Inversión - Financimmiento. Organismos Descentralizados. Base Caja.',
  'language': ['SPA'],
  '

Comprobamos que los metadatos de la API sean iguales a los originales

In [36]:
def compare_metadata(serie_meta, original_meta, equivalent_dict_keys):
    comparisons = []
    for key, value in equivalent_dict_keys.items():
        if isinstance(value, dict):
            for api_key, original_key in value.items():
                print(key, api_key, original_key)
                comparison = str(serie_meta[key][api_key]) == str(original_meta[original_key])
                comparisons.append(
                    '{} {} == {}: {} ({} {})'.format(key, api_key, original_key, comparison, serie_meta[key][api_key], original_meta[original_key]))
    return comparisons
    

In [39]:
keys = {
    'catalog': {
        'identifier': 'catalogo_id',
    },
    'dataset': {
        'identifier': 'dataset_id',
    },
    'distribution': {
        'identifier': 'distribucion_id',
        'accrualPeriodicity': 'indice_tiempo_frecuencia',
    },
    'field': {
        'id': 'serie_id',
    },
}

compare_metadata(serie_meta, orig_meta, keys)

catalog identifier catalogo_id
dataset identifier dataset_id
dataset accrualPeriodicity indice_tiempo_frecuencia
distribution identifier distribucion_id
field id serie_id


['catalog identifier == catalogo_id: True (sspm sspm)',
 'dataset identifier == dataset_id: True (375 375)',
 'dataset accrualPeriodicity == indice_tiempo_frecuencia: False (R/P1M R/P3M)',
 'distribution identifier == distribucion_id: True (375.4 375.4)',
 'field id == serie_id: True (375.4_GTOS_CAP_T006__32_51 375.4_GTOS_CAP_T006__32_51)']